<a href="https://colab.research.google.com/github/HaaziqKazi/DronesStuff/blob/main/Yolo_%2B_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip -q install kaggle

In [ ]:
from google.colab import files
files.upload()  # choose kaggle.json you downloaded

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"haaziqkazi1","key":"a5efed9a6f2e18d8c9a0c59781e173a4"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
SAVE_DIR = "/content/drive/MyDrive/kaggle_data"
!mkdir -p "$SAVE_DIR"

In [ ]:
DATASET = "andrewmvd/road-sign-detection"  # owner/dataset
!kaggle datasets download -d $DATASET -p "$SAVE_DIR" --force
!unzip -o "$SAVE_DIR/$(basename $DATASET).zip" -d "$SAVE_DIR"

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/road-sign-detection
License(s): CC0-1.0
 31% 67.0M/218M [00:03<00:07, 19.8MB/s]
User cancelled operation
Archive:  /content/drive/MyDrive/kaggle_data/road-sign-detection.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/drive/MyDrive/kaggle_data/road-sign-detection.zip or
        /content/drive/MyDrive/kaggle_data/road-sign-detection.zip.zip, and cannot find /content/drive/MyDrive/kaggle_data/road-sign-detection.zip.ZIP, period.


In [ ]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.1 MB/s eta 0:00:00


In [ ]:
import torch
from ultralytics import YOLO
import os, glob, re, random, shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
DATA_DIR = "/content/drive/MyDrive/kaggle_data"
IMG_DIR = os.path.join(DATA_DIR, "images")
ANN_DIR = os.path.join(DATA_DIR, "annotations")
LBL_DIR = os.path.join(DATA_DIR, "labels")
os.makedirs(LBL_DIR, exist_ok=True)

In [ ]:
def get_tag(text, tag):
    m = re.search(rf"<{tag}>\s*(.*?)\s*</{tag}>", text, re.DOTALL|re.IGNORECASE)
    return m.group(1).strip() if m else None

def get_tags(text, tag):
    return re.findall(rf"<{tag}>\s*(.*?)\s*</{tag}>", text, re.DOTALL|re.IGNORECASE)

def get_object_blocks(text):
    return re.findall(r"<object>(.*?)</object>", text, re.DOTALL|re.IGNORECASE)

In [ ]:
classes = []
for xml_path in sorted(glob.glob(os.path.join(ANN_DIR, "*.xml"))):
    with open(xml_path, "r", encoding="utf-8", errors="ignore") as f:
        x = f.read()
    for name in get_tags(x, "name"):
        name = name.strip()
        if name and name not in classes:
            classes.append(name)

classes.sort()
print("Classes:", classes)

Classes: ['crosswalk', 'speedlimit', 'stop', 'trafficlight']


In [ ]:
def voc_to_yolo(size_wh, box_xyxy):
    w, h = size_wh
    xmin, ymin, xmax, ymax = box_xyxy
    cx = (xmin + xmax) / 2.0 / w
    cy = (ymin + ymax) / 2.0 / h
    bw = (xmax - xmin) / w
    bh = (ymax - ymin) / h
    return cx, cy, bw, bh

bad = 0
converted = 0

for xml_path in sorted(glob.glob(os.path.join(ANN_DIR, "*.xml"))):
    with open(xml_path, "r", encoding="utf-8", errors="ignore") as f:
        x = f.read()

    filename = get_tag(x, "filename")
    w = get_tag(x, "width")
    h = get_tag(x, "height")

    if not filename or not w or not h:
        continue

    w, h = int(float(w)), int(float(h))
    stem = os.path.splitext(filename)[0]
    out_txt = os.path.join(LBL_DIR, stem + ".txt")

    lines = []
    for obj in get_object_blocks(x):
        name = get_tag(obj, "name")
        if name is None or name not in classes:
            continue
        cls_id = classes.index(name)

        try:
            xmin = float(get_tag(obj, "xmin"))
            ymin = float(get_tag(obj, "ymin"))
            xmax = float(get_tag(obj, "xmax"))
            ymax = float(get_tag(obj, "ymax"))
        except (TypeError, ValueError):
            bad += 1
            continue

        xmin, ymin = max(0, xmin), max(0, ymin)
        xmax, ymax = min(w-1, xmax), min(h-1, ymax)
        if xmax <= xmin or ymax <= ymin:
            bad += 1
            continue

        cx, cy, bw, bh = voc_to_yolo((w,h), (xmin,ymin,xmax,ymax))
        lines.append(f"{cls_id} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

    with open(out_txt, "w") as f:
        f.write("\n".join(lines))
    converted += 1

print(f"Converted {converted} xmls. Invalid/empty boxes skipped: {bad}")

Converted 877 xmls. Invalid/empty boxes skipped: 0


In [ ]:
YOLO_ROOT = "/content/yolo_data"
for s in ["train","val","test"]:
    os.makedirs(os.path.join(YOLO_ROOT, s, "images"), exist_ok=True)
    os.makedirs(os.path.join(YOLO_ROOT, s, "labels"), exist_ok=True)

# Pair images to labels by stem
pairs = []
for img_path in sorted(glob.glob(os.path.join(IMG_DIR, "*.*"))):
    stem = os.path.splitext(os.path.basename(img_path))[0]
    lbl_path = os.path.join(LBL_DIR, stem + ".txt")
    if os.path.exists(lbl_path):
        pairs.append((img_path, lbl_path))

random.shuffle(pairs)
n = len(pairs)
n_train = int(0.8 * n)
n_val   = int(0.1 * n)
train_pairs = pairs[:n_train]
val_pairs   = pairs[n_train:n_train+n_val]
test_pairs  = pairs[n_train+n_val:]

def copy_pairs(pairs, split):
    for img_p, lbl_p in pairs:
        shutil.copy(img_p, os.path.join(YOLO_ROOT, split, "images", os.path.basename(img_p)))
        shutil.copy(lbl_p, os.path.join(YOLO_ROOT, split, "labels", os.path.basename(lbl_p)))

copy_pairs(train_pairs, "train")
copy_pairs(val_pairs, "val")
copy_pairs(test_pairs, "test")

print("Counts -> train:", len(train_pairs), "val:", len(val_pairs), "test:", len(test_pairs))


Counts -> train: 701 val: 87 test: 89


In [ ]:
names_str = "[" + ", ".join([f"'{c}'" for c in classes]) + "]"
yaml_text = f"""path: {YOLO_ROOT}
train: train/images
val: val/images
test: test/images
names: {names_str}
"""
with open("/content/data.yaml", "w") as f:
    f.write(yaml_text)

!cat /content/data.yaml


path: /content/yolo_data
train: train/images
val: val/images
test: test/images
names: ['crosswalk', 'speedlimit', 'stop', 'trafficlight']


In [ ]:
from ultralytics import YOLO
import torch

print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
model = YOLO("yolov8n.pt")
model.train(data="/content/data.yaml", imgsz=640, epochs=30, batch=16, device=0 if torch.cuda.is_available() else 'cpu')


Torch: 2.8.0+cu126 | CUDA: True
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7edabb6d7470>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [ ]:
model.predict(source=os.path.join(YOLO_ROOT, "test", "images"), save=True, imgsz=640)


image 1/89 /content/yolo_data/test/images/road1.png: 480x640 1 speedlimit, 1 trafficlight, 39.6ms
image 2/89 /content/yolo_data/test/images/road110.png: 448x640 1 speedlimit, 38.9ms
image 3/89 /content/yolo_data/test/images/road135.png: 416x640 1 crosswalk, 40.0ms
image 4/89 /content/yolo_data/test/images/road155.png: 640x640 1 crosswalk, 7.9ms
image 5/89 /content/yolo_data/test/images/road156.png: 448x640 1 crosswalk, 1 trafficlight, 6.4ms
image 6/89 /content/yolo_data/test/images/road166.png: 640x480 1 crosswalk, 37.4ms
image 7/89 /content/yolo_data/test/images/road172.png: 640x480 1 speedlimit, 5.9ms
image 8/89 /content/yolo_data/test/images/road182.png: 640x480 1 speedlimit, 5.9ms
image 9/89 /content/yolo_data/test/images/road184.png: 640x480 1 speedlimit, 5.9ms
image 10/89 /content/yolo_data/test/images/road186.png: 640x480 1 crosswalk, 10.0ms
image 11/89 /content/yolo_data/test/images/road19.png: 640x448 1 trafficlight, 61.7ms
image 12/89 /content/yolo_data/test/images/road191.p

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'crosswalk', 1: 'speedlimit', 2: 'stop', 3: 'trafficlight'}
 obb: None
 orig_img: array([[[212, 191, 178],
         [212, 191, 178],
         [213, 192, 178],
         ...,
         [215, 197, 183],
         [215, 197, 183],
         [215, 197, 183]],
 
        [[212, 191, 178],
         [212, 191, 178],
         [213, 192, 179],
         ...,
         [216, 197, 183],
         [215, 197, 183],
         [215, 197, 183]],
 
        [[212, 191, 178],
         [212, 191, 178],
         [213, 192, 179],
         ...,
         [216, 197, 183],
         [215, 197, 183],
         [215, 197, 183]],
 
        ...,
 
        [[ 14,  12,  15],
         [ 18,  12,  14],
         [ 43,  34,  32],
         ...,
         [136, 117, 108],
         [118, 107, 104],
         [112, 106, 105]],
 
        [[ 13,  11,  15],
         [ 19,  14,  16],
        